In [1]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor

In [75]:
anos = [2017, 2020, 2024]
deltas = ['2020-2017', '2024-2020']

gdf_favelas = gpd.read_file('data/SIRGAS_GPKG_favela.gpkg')

BMHs = [f'../LiDAR_produtos/{i}/BHM-{i}-50cm.tiff' for i in anos] # 50cm
MDTs = [f'../LiDAR_produtos/{i}/MDT-{i}-50cm.tiff' for i in anos] # 50cm
MDSs_sem_veg = [f'../LiDAR_produtos/{i}/MDS_sem_vegetacao-{i}-1m-preenchido.tiff' for i in anos] # 1m
MDSs_sem_veg_delta = [f'../LiDAR_produtos/{i}/MDS_sem_vegetacao-{i}-1m-preenchido.tiff' for i in deltas] # 1m

rasters = BMHs + MDTs + MDSs_sem_veg + MDSs_sem_veg_delta


/Users/fernandogomes/miniconda3/envs/geo/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(


In [76]:
os.path.splitext(os.path.basename(rasters[-1]))[0] 

'MDS_sem_vegetacao-2024-2020-1m-preenchido'

In [77]:
# gdf_favelas = gdf_favelas.head(1)

In [78]:
def process_raster(raster_path):
    stats = zonal_stats(
        vectors=gdf_favelas,
        raster=raster_path,
        stats=["count", "min", "max", "median", "std", "percentile_25", "percentile_75"],
        # nodata=-9999
    )
    df = pd.DataFrame(stats)
    # Nomear colunas com prefixo do nome do raster
    raster_name = os.path.splitext(os.path.basename(raster_path))[0] 
    df = df.add_prefix(f"{raster_name}_")
    return df

In [79]:
# Processar usando threads
with ThreadPoolExecutor(max_workers=12) as executor:
    results = list(executor.map(process_raster, rasters))

/Users/fernandogomes/miniconda3/envs/geo/lib/python3.12/site-packages/rasterio/features.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)


In [80]:
# Juntar resultados
stats_all = pd.concat(results, axis=1)
gdf_favelas = gdf_favelas.join(stats_all)

In [81]:
gdf_favelas

,fv_nome,fv_data,fv_nom_sec,fv_tp_prop,fv_domicil,geometry,BHM-2017-50cm_min,BHM-2017-50cm_max,BHM-2017-50cm_count,BHM-2017-50cm_std,...,MDS_sem_vegetacao-2020-2017-1m-preenchido_median,MDS_sem_vegetacao-2020-2017-1m-preenchido_percentile_25,MDS_sem_vegetacao-2020-2017-1m-preenchido_percentile_75,MDS_sem_vegetacao-2024-2020-1m-preenchido_min,MDS_sem_vegetacao-2024-2020-1m-preenchido_max,MDS_sem_vegetacao-2024-2020-1m-preenchido_count,MDS_sem_vegetacao-2024-2020-1m-preenchido_std,MDS_sem_vegetacao-2024-2020-1m-preenchido_median,MDS_sem_vegetacao-2024-2020-1m-preenchido_percentile_25,MDS_sem_vegetacao-2024-2020-1m-preenchido_percentile_75
0,OCUPAÇÃO PROMORAR - São Francisco Global,2016-06-30,-,1,1000,"POLYGON ((3.51e+05 7.39e+06, 3.51e+05 7.39e+06...",0.343474,14.606697,114773,1.371275,...,0.066528,-0.141907,0.733612,-4.676636,10.859863,39856,1.298460,0.534088,0.255676,1.314621
1,Jardim Solange,2016-04-20,Jardim Raniere,1,180,"POLYGON ((3.18e+05 7.38e+06, 3.18e+05 7.38e+06...",0.003708,19.656366,110215,3.037627,...,-0.155060,-0.861099,0.059021,-13.282349,16.798828,43253,1.485442,0.422729,0.141418,1.148071
2,Recanto do Sol,2016-04-19,-,1,29,"POLYGON ((3.33e+05 7.37e+06, 3.33e+05 7.37e+06...",0.073508,8.358132,13969,1.089970,...,-0.260010,-0.545914,-0.012924,-12.306519,6.024780,14129,1.079694,0.370972,0.035339,0.626099
3,Beira Rio,2016-04-04,Hélio Lobo,1,659,"POLYGON ((3.31e+05 7.38e+06, 3.31e+05 7.38e+06...",1.640960,9.040278,4999,1.630700,...,-0.075745,-0.228897,0.083237,-1.887146,6.439758,1389,1.255357,0.464478,0.195801,1.179565
4,Beira Rio,2016-04-04,Hélio Lobo,1,659,"POLYGON ((3.31e+05 7.38e+06, 3.31e+05 7.38e+06...",1.100818,9.796526,36045,1.728493,...,-0.063599,-0.164551,0.364441,-6.147583,11.635620,16823,1.642416,0.291992,0.104523,0.959930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,Mirangoaba II,2016-04-14,Penteado,1,387,"POLYGON ((3.28e+05 7.4e+06, 3.28e+05 7.4e+06, ...",0.035901,13.952505,49924,2.277910,...,-0.115845,-0.329147,0.151962,-9.481201,12.709045,16085,1.463644,0.546448,0.150574,1.412964
2159,Mirangoaba II,2016-04-14,Penteado,1,387,"POLYGON ((3.28e+05 7.4e+06, 3.28e+05 7.4e+06, ...",0.272292,11.024065,4506,2.086676,...,-0.087524,-0.196167,0.173676,-2.069519,3.796387,1247,0.703005,0.326172,0.113495,0.856384
2160,Hamilton de Araújo,2016-04-12,Bananal do Rio,1,21,"POLYGON ((3.21e+05 7.41e+06, 3.21e+05 7.41e+06...",1.966069,11.874878,4726,2.189462,...,-0.112457,-0.288483,0.116486,-3.583984,6.129517,1298,1.719061,0.683136,0.018494,2.071671
2161,Jardim Dos Francos II,2016-04-14,Laudelino Antônio de Brito,1,22,"POLYGON ((3.29e+05 7.41e+06, 3.29e+05 7.41e+06...",0.863413,12.651338,4919,2.811603,...,-0.130249,-0.365417,0.072754,-1.422913,4.078308,1329,0.744831,0.409485,0.133057,1.020386


In [84]:
gdf_favelas.area.median()

3734.719928381712